In [10]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

# Now import tensorflow/keras modules
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np

In [11]:
import os 
import re

image_dir = 'BBBC005_v1_images'
files = []
labels = []

for file in os.listdir(image_dir):
    if file.endswith('.TIF'):
        files.append(os.path.join(image_dir, file))
        labels.append(int(re.findall('_C(.*?)_', file)[0]))
dataset = dc.data.ImageDataset(files, np.array(labels))


In [12]:
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset, seed=123)

In [13]:
features = tf.keras.Input(shape = (520,696,1))
prev_layer = features
for num_outputs in [16,32,64,128,256]:
    prev_layer = layers.Conv2D(num_outputs, kernel_size=5, strides=2,
                               activation=tf.nn.relu)(prev_layer)
output = layers.Dense(1)(layers.Flatten()(prev_layer))

keras_model = tf.keras.Model(inputs=features, outputs=output)
learning_rate = dc.models.optimizers.ExponentialDecay(0.001, 0.9, 250)
model = dc.models.KerasModel(
    keras_model,
    loss = dc.models.losses.L2Loss(),
    learning_rate = learning_rate,
    model_dir='model'
)



In [14]:
model.restore()

In [15]:
y_pred = model.predict(test_dataset).flatten()
print(np.sqrt(np.mean((y_pred-test_dataset.y)**2)))


57.262351802321085


In [16]:
#model.fit(train_dataset, nb_epoch=50)

Cell segmentation

In [17]:
image_dir = 'BBBC005_v1_images'
label_dir = 'BBBC005_v1_ground_truth'
rows = {'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'}
blurs = (1,4,7,10,14,17,20,23,26,29,32,35,39,42,45,48)
files = []
labels = []
for f in os.listdir(label_dir):
    if f.endswith('.TIF'):
        for row, blur in zip(row, blurs):
            fname - f.replace('_F1','_F%d'%blur).replace('_A','_%s'%row)
            files.append(os.path.join(image_dir, fname))
            labels.append(os.path.join(label_dir, f))
dataset = dc.data.ImageDataset(files, labels)

NameError: name 'row' is not defined